In [1]:
import requests
import toml
from pprint import pprint
from sys import argv, exit
from os import path

In [2]:
# Load N2YO API key from file (get your own key by registering at N2YO.com)
if path.exists("N2YO_API_KEY.txt"):
    API_KEY = open("N2YO_API_KEY.txt").read().strip()
else:
    print("Error: API key file (N2YO_API_KEY.txt) not found!")
    exit()

In [3]:
# Load basic query parameters from 'config.toml'
if path.exists("config.toml"):
    base_params = toml.loads(open("config.toml").read())
else:
    print("Error: config.toml not found!")
    exit()

In [4]:
# URL templates to the N2YO API, used to produce the query URL below
BASE_URL = "https://www.n2yo.com/rest/v1/satellite/"
GET_TLE = "tle/{id}"
VISUAL_PASSES = "visualpasses/{id:d}/{observer_lat:.5f}/{observer_lng:.5f}/{observer_alt:2f}/{days:d}/{min_visibility:d}"
SAT_POSITION = "positions/{id}/{observer_lat}/{observer_lng}/{observer_alt}/{seconds}"
RADIO_PASSES = "radiopasses/{id}/{observer_lat}/{observer_lng}/{observer_alt}/{days}/{min_elevation}"

In [5]:
def parse_query(params, template_name, debug=False):
    " Produce URL from parameter dictionary."
    URL = "".join([BASE_URL, template_name.format(**params)])
    if debug:
        print("Created query: {}".format(URL))
    return URL


In [6]:
def retrieve_data(QUERY_URL, debug=False):
    "Retrieve data from given URL."
    if debug:
        print("Requesting data from: {}".format(QUERY_URL))
    r = requests.get(QUERY_URL, params={"apiKey" : API_KEY})
    if r.status_code == requests.codes.ok:
        if debug:
            print("Success.")
        return r.json()
    elif debug:
        print("Failed! (status code {})".format(r.status_code))
    return "{}"

In [7]:
def read_ids(filename, debug=False):
    "Read a list of satellite ID numbers from given filename."
    IDs = []
    if debug:
        print("Parsing file: {}".format(filename))
    with open(filename, "r") as f:
        for line in f:
            ID = int(line.strip())
            if debug:
                print(ID)
            IDs.append(ID)
    return IDs

In [8]:
def get_single_visible(ID, debug=False):
    "Retrieve data of a single satellite based on ID."
    query_params = base_params.copy()
    query_params["id"] = ID
    if debug:
        print("Query parameters:")
        pprint(query_params)
    URL = parse_query(query_params, VISUAL_PASSES,debug)
    data = retrieve_data(URL, debug)
    return data

In [9]:
def get_single_satposition(ID, debug=False):
    "Retrieve data of a single satellite based on ID."
    query_params = base_params.copy()
    query_params["id"] = ID
    if debug:
        print("Query parameters:")
        pprint(query_params)
    URL = parse_query(query_params, SAT_POSITION,debug)
    data = retrieve_data(URL, debug)
    return data

In [10]:
def get_single_radpasses(ID, debug=False):
    "Retrieve data of a single satellite based on ID."
    query_params = base_params.copy()
    query_params["id"] = ID
    if debug:
        print("Query parameters:")
        pprint(query_params)
    URL = parse_query(query_params, RADIO_PASSES,debug)
    data = retrieve_data(URL, debug)
    return data

In [11]:
def get_single_tle(ID, debug=False):
    "Retrieve data of a single satellite based on ID."
    query_params = base_params.copy()
    query_params["id"] = ID
    if debug:
        print("Query parameters:")
        pprint(query_params)
    URL = parse_query(query_params, RADIO_PASSES,debug)
    data = retrieve_data(URL, debug)
    return data

In [12]:
def get_all_visible(filename, debug=False):
    "Retrieve data of all satellites in a list of IDs."
    IDs = read_ids(filename, debug)
    all_data = [get_single_visible(ID, debug) for ID in IDs]
    return all_data

In [13]:
def get_all_positions(filename, debug=False):
    "Retrieve data of all satellites positions in a list of IDs."
    IDs = read_ids(filename, debug)
    all_data = [get_single_satposition(ID, debug) for ID in IDs]
    return all_data

In [14]:
def get_all_radiopositions(filename, debug=False):
    "Retrieve data of all satellites radio passes in a list of IDs."
    IDs = read_ids(filename, debug)
    all_data = [get_single_radpasses(ID, debug) for ID in IDs]
    return all_data

In [15]:
def get_all_tle(filename, debug=False):
    "Retrieve tle of all satellites radio passes in a list of IDs."
    IDs = read_ids(filename, debug)
    all_data = [get_single_tle(ID, debug) for ID in IDs]
    return all_data

In [16]:
if __name__=="__main__":
    fname = "test_list.txt" #argv[1]

In [17]:
#get the TLE for a satellite
tle = get_all_tle(fname, debug=True)
for result in tle:
    pprint(result)

Parsing file: test_list.txt
25544
Query parameters:
{u'days': 1,
 'id': 25544,
 u'min_elevation': 60,
 u'min_visibility': 60,
 u'observer_alt': 14.003,
 u'observer_lat': 19.076,
 u'observer_lng': 72.8777,
 u'seconds': 3600}
Created query: https://www.n2yo.com/rest/v1/satellite/radiopasses/25544/19.076/72.8777/14.003/1/60
Requesting data from: https://www.n2yo.com/rest/v1/satellite/radiopasses/25544/19.076/72.8777/14.003/1/60
Success.
{u'info': {u'passescount': 0,
           u'satid': 25544,
           u'satname': u'SPACE STATION',
           u'transactionscount': 12}}


In [19]:
#get data for all satellites in the list
data = get_all_visible(fname, debug=True)
for result in data:
    pprint(result)

Parsing file: test_list.txt
25544
Query parameters:
{u'days': 1,
 'id': 25544,
 u'min_elevation': 60,
 u'min_visibility': 60,
 u'observer_alt': 14.003,
 u'observer_lat': 19.076,
 u'observer_lng': 72.8777,
 u'seconds': 3600}
Created query: https://www.n2yo.com/rest/v1/satellite/visualpasses/25544/19.07600/72.87770/14.003000/1/60
Requesting data from: https://www.n2yo.com/rest/v1/satellite/visualpasses/25544/19.07600/72.87770/14.003000/1/60
Success.
{u'info': {u'passescount': 1,
           u'satid': 25544,
           u'satname': u'SPACE STATION',
           u'transactionscount': 13},
 u'passes': [{u'duration': 190,
              u'endAz': 33.21,
              u'endAzCompass': u'NE',
              u'endEl': 0,
              u'endUTC': 1540597030,
              u'mag': -0.5,
              u'maxAz': 312.72,
              u'maxAzCompass': u'NW',
              u'maxEl': 54.24,
              u'maxUTC': 1540596720,
              u'startAz': 225.91,
              u'startAzCompass': u'SW',
      

In [20]:
#get positions for all satellites in the list
pos = get_all_positions(fname, debug=True)
for positions in pos:
    pprint(positions)

Parsing file: test_list.txt
25544
Query parameters:
{u'days': 1,
 'id': 25544,
 u'min_elevation': 60,
 u'min_visibility': 60,
 u'observer_alt': 14.003,
 u'observer_lat': 19.076,
 u'observer_lng': 72.8777,
 u'seconds': 3600}
Created query: https://www.n2yo.com/rest/v1/satellite/positions/25544/19.076/72.8777/14.003/3600
Requesting data from: https://www.n2yo.com/rest/v1/satellite/positions/25544/19.076/72.8777/14.003/3600
Success.
{u'info': {u'satid': 25544,
           u'satname': u'SPACE STATION',
           u'transactionscount': 14},
 u'positions': [{u'azimuth': 322.07,
                 u'dec': -11.31054508,
                 u'elevation': -80.21,
                 u'ra': 76.05928078,
                 u'sataltitude': 404.24,
                 u'satlatitude': -3.94265473,
                 u'satlongitude': -95.55105949,
                 u'timestamp': 1540546222},
                {u'azimuth': 322.07,
                 u'dec': -11.28456081,
                 u'elevation': -80.17,
             

                 u'satlatitude': 2.2956406,
                 u'satlongitude': -91.15049971,
                 u'timestamp': 1540546344},
                {u'azimuth': 322.13,
                 u'dec': -8.10365579,
                 u'elevation': -76.25,
                 u'ra': 78.92136506,
                 u'sataltitude': 403.49,
                 u'satlatitude': 2.34677173,
                 u'satlongitude': -91.11440794,
                 u'timestamp': 1540546345},
                {u'azimuth': 322.13,
                 u'dec': -8.0775076,
                 u'elevation': -76.22,
                 u'ra': 78.94446773,
                 u'sataltitude': 403.49,
                 u'satlatitude': 2.39790173,
                 u'satlongitude': -91.07831319,
                 u'timestamp': 1540546346},
                {u'azimuth': 322.13,
                 u'dec': -8.05135734,
                 u'elevation': -76.19,
                 u'ra': 78.96756916,
                 u'sataltitude': 403.49,
               

                 u'ra': 80.7866921,
                 u'sataltitude': 403.27,
                 u'satlatitude': 6.48230487,
                 u'satlongitude': -88.17540885,
                 u'timestamp': 1540546426},
                {u'azimuth': 322.15,
                 u'dec': -5.95658318,
                 u'elevation': -73.61,
                 u'ra': 80.80965822,
                 u'sataltitude': 403.26,
                 u'satlatitude': 6.53325455,
                 u'satlongitude': -88.13886007,
                 u'timestamp': 1540546427},
                {u'azimuth': 322.15,
                 u'dec': -5.93036786,
                 u'elevation': -73.58,
                 u'ra': 80.83262408,
                 u'sataltitude': 403.26,
                 u'satlatitude': 6.58420294,
                 u'satlongitude': -88.10230155,
                 u'timestamp': 1540546428},
                {u'azimuth': 322.15,
                 u'dec': -5.90415406,
                 u'elevation': -73.55,
              

                 u'satlongitude': -85.07015994,
                 u'timestamp': 1540546510},
                {u'azimuth': 322.15,
                 u'dec': -3.75297942,
                 u'elevation': -70.91,
                 u'ra': 82.73578203,
                 u'sataltitude': 403.25,
                 u'satlatitude': 10.79836717,
                 u'satlongitude': -85.03268285,
                 u'timestamp': 1540546511},
                {u'azimuth': 322.15,
                 u'dec': -3.7267328,
                 u'elevation': -70.88,
                 u'ra': 82.75869056,
                 u'sataltitude': 403.25,
                 u'satlatitude': 10.8489325,
                 u'satlongitude': -84.99519333,
                 u'timestamp': 1540546512},
                {u'azimuth': 322.15,
                 u'dec': -3.700486,
                 u'elevation': -70.85,
                 u'ra': 82.78159894,
                 u'sataltitude': 403.25,
                 u'satlatitude': 10.89949206,
              

                 u'elevation': -68.04,
                 u'ra': 84.77544873,
                 u'sataltitude': 403.46,
                 u'satlatitude': 15.27267683,
                 u'satlongitude': -81.63321695,
                 u'timestamp': 1540546600},
                {u'azimuth': 322.13,
                 u'dec': -1.39069104,
                 u'elevation': -68.01,
                 u'ra': 84.7983928,
                 u'sataltitude': 403.46,
                 u'satlatitude': 15.3226109,
                 u'satlongitude': -81.59419983,
                 u'timestamp': 1540546601},
                {u'azimuth': 322.13,
                 u'dec': -1.36444964,
                 u'elevation': -67.98,
                 u'ra': 84.8213369,
                 u'sataltitude': 403.47,
                 u'satlatitude': 15.3725344,
                 u'satlongitude': -81.55516382,
                 u'timestamp': 1540546602},
                {u'azimuth': 322.13,
                 u'dec': -1.33820853,
              

...

                 u'satlongitude': 86.9451247,
                 u'timestamp': 1540549385},
                {u'azimuth': 157.2,
                 u'dec': -67.54540874,
                 u'elevation': -13.46,
                 u'ra': 341.98102173,
                 u'sataltitude': 411.57,
                 u'satlatitude': -15.7541155,
                 u'satlongitude': 86.98421671,
                 u'timestamp': 1540549386},
                {u'azimuth': 157.18,
                 u'dec': -67.53265962,
                 u'elevation': -13.5,
                 u'ra': 342.10318867,
                 u'sataltitude': 411.58,
                 u'satlatitude': -15.80384472,
                 u'satlongitude': 87.02332963,
                 u'timestamp': 1540549387},
                {u'azimuth': 157.15,
                 u'dec': -67.51989008,
                 u'elevation': -13.53,
                 u'ra': 342.2251151,
                 u'sataltitude': 411.59,
                 u'satlatitude': -15.85356499,
         

                {u'azimuth': 154.68,
                 u'dec': -66.06246135,
                 u'elevation': -17.56,
                 u'ra': 353.96417528,
                 u'sataltitude': 413,
                 u'satlatitude': -21.06607589,
                 u'satlongitude': 91.34577969,
                 u'timestamp': 1540549494},
                {u'azimuth': 154.66,
                 u'dec': -66.04781034,
                 u'elevation': -17.59,
                 u'ra': 354.06516222,
                 u'sataltitude': 413.01,
                 u'satlatitude': -21.11463457,
                 u'satlongitude': 91.38761912,
                 u'timestamp': 1540549495},
                {u'azimuth': 154.64,
                 u'dec': -66.03314345,
                 u'elevation': -17.63,
                 u'ra': 354.16598915,
                 u'sataltitude': 413.03,
                 u'satlatitude': -21.16318037,
                 u'satlongitude': 91.42948833,
                 u'timestamp': 1540549496},
       

                {u'azimuth': 153.02,
                 u'dec': -64.73001527,
                 u'elevation': -20.72,
                 u'ra': 2.20081948,
                 u'sataltitude': 414.25,
                 u'satlatitude': -25.23815443,
                 u'satlongitude': 95.10667053,
                 u'timestamp': 1540549581},
                {u'azimuth': 153,
                 u'dec': -64.7140361,
                 u'elevation': -20.75,
                 u'ra': 2.2895568,
                 u'sataltitude': 414.26,
                 u'satlatitude': -25.28543404,
                 u'satlongitude': 95.15143543,
                 u'timestamp': 1540549582},
                {u'azimuth': 152.98,
                 u'dec': -64.69804085,
                 u'elevation': -20.79,
                 u'ra': 2.37817729,
                 u'sataltitude': 414.28,
                 u'satlatitude': -25.33270082,
                 u'satlongitude': 95.19624168,
                 u'timestamp': 1540549583},
               

                 u'elevation': -23.55,
                 u'ra': 8.93278987,
                 u'sataltitude': 415.43,
                 u'satlatitude': -28.964091,
                 u'satlongitude': 98.81550715,
                 u'timestamp': 1540549661},
                {u'azimuth': 151.66,
                 u'dec': -63.38842685,
                 u'elevation': -23.58,
                 u'ra': 9.01252672,
                 u'sataltitude': 415.45,
                 u'satlatitude': -29.00988609,
                 u'satlongitude': 98.86361303,
                 u'timestamp': 1540549662},
                {u'azimuth': 151.64,
                 u'dec': -63.37127671,
                 u'elevation': -23.62,
                 u'ra': 9.09215488,
                 u'sataltitude': 415.46,
                 u'satlatitude': -29.05565676,
                 u'satlongitude': 98.91176094,
                 u'timestamp': 1540549663},
                {u'azimuth': 151.63,
                 u'dec': -63.35411109,
            

                {u'azimuth': 150.11,
                 u'dec': -61.54941758,
                 u'elevation': -27.14,
                 u'ra': 16.71463765,
                 u'sataltitude': 416.98,
                 u'satlatitude': -33.60479071,
                 u'satlongitude': 104.08408723,
                 u'timestamp': 1540549765},
                {u'azimuth': 150.09,
                 u'dec': -61.53086264,
                 u'elevation': -27.18,
                 u'ra': 16.78480091,
                 u'sataltitude': 416.99,
                 u'satlatitude': -33.64810643,
                 u'satlongitude': 104.13755682,
                 u'timestamp': 1540549766},
                {u'azimuth': 150.08,
                 u'dec': -61.51229609,
                 u'elevation': -27.21,
                 u'ra': 16.85487569,
                 u'sataltitude': 417.01,
                 u'satlatitude': -33.69139141,
                 u'satlongitude': 104.19107963,
                 u'timestamp': 1540549767},
    

In [21]:
#get radio positions for all satellites in the list
radio = get_all_radiopositions(fname, debug=True)
for positions in pos:
    pprint(positions)

Parsing file: test_list.txt
25544
Query parameters:
{u'days': 1,
 'id': 25544,
 u'min_elevation': 60,
 u'min_visibility': 60,
 u'observer_alt': 14.003,
 u'observer_lat': 19.076,
 u'observer_lng': 72.8777,
 u'seconds': 3600}
Created query: https://www.n2yo.com/rest/v1/satellite/radiopasses/25544/19.076/72.8777/14.003/1/60
Requesting data from: https://www.n2yo.com/rest/v1/satellite/radiopasses/25544/19.076/72.8777/14.003/1/60
Success.
{u'info': {u'satid': 25544,
           u'satname': u'SPACE STATION',
           u'transactionscount': 14},
 u'positions': [{u'azimuth': 322.07,
                 u'dec': -11.31054508,
                 u'elevation': -80.21,
                 u'ra': 76.05928078,
                 u'sataltitude': 404.24,
                 u'satlatitude': -3.94265473,
                 u'satlongitude': -95.55105949,
                 u'timestamp': 1540546222},
                {u'azimuth': 322.07,
                 u'dec': -11.28456081,
                 u'elevation': -80.17,
         

                 u'elevation': -76.22,
                 u'ra': 78.94446773,
                 u'sataltitude': 403.49,
                 u'satlatitude': 2.39790173,
                 u'satlongitude': -91.07831319,
                 u'timestamp': 1540546346},
                {u'azimuth': 322.13,
                 u'dec': -8.05135734,
                 u'elevation': -76.19,
                 u'ra': 78.96756916,
                 u'sataltitude': 403.49,
                 u'satlatitude': 2.44903264,
                 u'satlongitude': -91.04221395,
                 u'timestamp': 1540546347},
                {u'azimuth': 322.13,
                 u'dec': -8.02520818,
                 u'elevation': -76.16,
                 u'ra': 78.99066656,
                 u'sataltitude': 403.48,
                 u'satlatitude': 2.50015825,
                 u'satlongitude': -91.00611451,
                 u'timestamp': 1540546348},
                {u'azimuth': 322.13,
                 u'dec': -7.99905592,
             

                 u'dec': -5.51086002,
                 u'elevation': -73.07,
                 u'ra': 81.1999132,
                 u'sataltitude': 403.24,
                 u'satlatitude': 7.39887185,
                 u'satlongitude': -87.51620845,
                 u'timestamp': 1540546444},
                {u'azimuth': 322.15,
                 u'dec': -5.48463633,
                 u'elevation': -73.03,
                 u'ra': 81.22285979,
                 u'sataltitude': 403.24,
                 u'satlatitude': 7.4497572,
                 u'satlongitude': -87.47950087,
                 u'timestamp': 1540546445},
                {u'azimuth': 322.15,
                 u'dec': -5.45841212,
                 u'elevation': -73,
                 u'ra': 81.24580542,
                 u'sataltitude': 403.24,
                 u'satlatitude': 7.50063871,
                 u'satlongitude': -87.44278384,
                 u'timestamp': 1540546446},
                {u'azimuth': 322.15,
                 u

                 u'sataltitude': 403.27,
                 u'satlatitude': 11.60670815,
                 u'satlongitude': -84.43112881,
                 u'timestamp': 1540546527},
                {u'azimuth': 322.15,
                 u'dec': -3.3067651,
                 u'elevation': -70.36,
                 u'ra': 83.12521648,
                 u'sataltitude': 403.27,
                 u'satlatitude': 11.65717768,
                 u'satlongitude': -84.39340694,
                 u'timestamp': 1540546528},
                {u'azimuth': 322.15,
                 u'dec': -3.28051628,
                 u'elevation': -70.33,
                 u'ra': 83.14812408,
                 u'sataltitude': 403.27,
                 u'satlatitude': 11.70764096,
                 u'satlongitude': -84.35566996,
                 u'timestamp': 1540546529},
                {u'azimuth': 322.15,
                 u'dec': -3.25426738,
                 u'elevation': -70.3,
                 u'ra': 83.17103174,
            

                {u'azimuth': 322.13,
                 u'dec': -1.18076727,
                 u'elevation': -67.75,
                 u'ra': 84.98197407,
                 u'sataltitude': 403.49,
                 u'satlatitude': 15.72175872,
                 u'satlongitude': -81.28133081,
                 u'timestamp': 1540546609},
                {u'azimuth': 322.13,
                 u'dec': -1.15452836,
                 u'elevation': -67.72,
                 u'ra': 85.0049262,
                 u'sataltitude': 403.5,
                 u'satlatitude': 15.77161262,
                 u'satlongitude': -81.24212835,
                 u'timestamp': 1540546610},
                {u'azimuth': 322.13,
                 u'dec': -1.12828978,
                 u'elevation': -67.69,
                 u'ra': 85.02787938,
                 u'sataltitude': 403.5,
                 u'satlatitude': 15.82145766,
                 u'satlongitude': -81.20290474,
                 u'timestamp': 1540546611},
             

...

                {u'azimuth': 157.18,
                 u'dec': -67.53265962,
                 u'elevation': -13.5,
                 u'ra': 342.10318867,
                 u'sataltitude': 411.58,
                 u'satlatitude': -15.80384472,
                 u'satlongitude': 87.02332963,
                 u'timestamp': 1540549387},
                {u'azimuth': 157.15,
                 u'dec': -67.51989008,
                 u'elevation': -13.53,
                 u'ra': 342.2251151,
                 u'sataltitude': 411.59,
                 u'satlatitude': -15.85356499,
                 u'satlongitude': 87.06246354,
                 u'timestamp': 1540549388},
                {u'azimuth': 157.12,
                 u'dec': -67.50709969,
                 u'elevation': -13.57,
                 u'ra': 342.34680692,
                 u'sataltitude': 411.6,
                 u'satlatitude': -15.90327827,
                 u'satlongitude': 87.10162009,
                 u'timestamp': 1540549389},
       

                {u'azimuth': 154.81,
                 u'dec': -66.15003373,
                 u'elevation': -17.33,
                 u'ra': 353.35485968,
                 u'sataltitude': 412.92,
                 u'satlatitude': -20.77445376,
                 u'satlongitude': 91.09536175,
                 u'timestamp': 1540549488},
                {u'azimuth': 154.79,
                 u'dec': -66.13547832,
                 u'elevation': -17.37,
                 u'ra': 353.45681783,
                 u'sataltitude': 412.93,
                 u'satlatitude': -20.82308885,
                 u'satlongitude': 91.13702437,
                 u'timestamp': 1540549489},
                {u'azimuth': 154.77,
                 u'dec': -66.12090694,
                 u'elevation': -17.41,
                 u'ra': 353.55861277,
                 u'sataltitude': 412.94,
                 u'satlatitude': -20.8717113,
                 u'satlongitude': 91.17871624,
                 u'timestamp': 1540549490},
     

                 u'satlongitude': 95.46586681,
                 u'timestamp': 1540549589},
                {u'azimuth': 152.86,
                 u'dec': -64.58566429,
                 u'elevation': -21.04,
                 u'ra': 2.99505563,
                 u'sataltitude': 414.38,
                 u'satlatitude': -25.66308706,
                 u'satlongitude': 95.51094012,
                 u'timestamp': 1540549590},
                {u'azimuth': 152.84,
                 u'dec': -64.56955161,
                 u'elevation': -21.08,
                 u'ra': 3.08269246,
                 u'sataltitude': 414.39,
                 u'satlatitude': -25.71021714,
                 u'satlongitude': 95.55605015,
                 u'timestamp': 1540549591},
                {u'azimuth': 152.83,
                 u'dec': -64.5534242,
                 u'elevation': -21.11,
                 u'ra': 3.1702082,
                 u'sataltitude': 414.41,
                 u'satlatitude': -25.75733016,
            

                 u'ra': 10.66365418,
                 u'sataltitude': 415.76,
                 u'satlatitude': -29.96667079,
                 u'satlongitude': 99.88456783,
                 u'timestamp': 1540549683},
                {u'azimuth': 151.31,
                 u'dec': -63.00786215,
                 u'elevation': -24.35,
                 u'ra': 10.74118843,
                 u'sataltitude': 415.78,
                 u'satlatitude': -30.01199474,
                 u'satlongitude': 99.93370638,
                 u'timestamp': 1540549684},
                {u'azimuth': 151.3,
                 u'dec': -62.99040232,
                 u'elevation': -24.39,
                 u'ra': 10.81862827,
                 u'sataltitude': 415.79,
                 u'satlatitude': -30.05729851,
                 u'satlongitude': 99.98289518,
                 u'timestamp': 1540549685},
                {u'azimuth': 151.28,
                 u'dec': -62.97292928,
                 u'elevation': -24.42,
        

                 u'timestamp': 1540549785},
                {u'azimuth': 149.81,
                 u'dec': -61.15704171,
                 u'elevation': -27.86,
                 u'ra': 18.17082232,
                 u'sataltitude': 417.28,
                 u'satlatitude': -34.50857219,
                 u'satlongitude': 105.21914432,
                 u'timestamp': 1540549786},
                {u'azimuth': 149.79,
                 u'dec': -61.13821482,
                 u'elevation': -27.89,
                 u'ra': 18.23927509,
                 u'sataltitude': 417.3,
                 u'satlatitude': -34.55129893,
                 u'satlongitude': 105.27384417,
                 u'timestamp': 1540549787},
                {u'azimuth': 149.78,
                 u'dec': -61.11937507,
                 u'elevation': -27.93,
                 u'ra': 18.30764819,
                 u'sataltitude': 417.31,
                 u'satlatitude': -34.59399691,
                 u'satlongitude': 105.32860399,
     